In [4]:
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo

In [5]:
import torch

In [28]:
__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        print('After conv1:', out.shape)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def resnet50(pretrained=False, **kwargs):
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model

def resnet50factorized(pretrained=False, **kwargs):
    model = ResNet(BottleneckFactorized, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model


In [41]:
class FConv3(nn.Module):
    def __init__(self, planes, stride):
        super().__init__()
        self.relu = nn.ReLU(inplace=True)
        self.stride = stride
        if self.stride == 1:
            self.conv = nn.Conv2d(planes, planes, kernel_size=(1,3), stride=stride,
                                   padding=(0,1), bias=False)
            self.bn = nn.BatchNorm2d(planes)
            self.conv1 = nn.Conv2d(planes, planes, kernel_size=(3,1), stride=stride,
                                   padding=(1,0), bias=False)
            self.bn1 = nn.BatchNorm2d(planes)
        else:
            self.conv = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                                   padding=1, bias=False)
            self.bn = nn.BatchNorm2d(planes)
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        if self.stride == 1:
            out = self.conv1(out)
            out = self.bn1(out)
            out = self.relu(out)
        return out
            

class BottleneckFactorized(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BottleneckFactorized, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = FConv3(planes, stride)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        print('Stride:', stride)

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        print('After conv1', out.shape)
        out = self.conv2(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out



In [42]:
m = resnet50factorized().cuda()

Stride: 1
Stride: 1
Stride: 1
Stride: 2
Stride: 1
Stride: 1
Stride: 1
Stride: 2
Stride: 1
Stride: 1
Stride: 1
Stride: 1
Stride: 1
Stride: 2
Stride: 1
Stride: 1


In [43]:
x = torch.ones((64,3,128,128)).float().cuda()

In [44]:
m(x)

After conv1 torch.Size([64, 64, 32, 32])
After conv1 torch.Size([64, 64, 32, 32])
After conv1 torch.Size([64, 64, 32, 32])
After conv1 torch.Size([64, 128, 32, 32])
After conv1 torch.Size([64, 128, 16, 16])
After conv1 torch.Size([64, 128, 16, 16])
After conv1 torch.Size([64, 128, 16, 16])
After conv1 torch.Size([64, 256, 16, 16])
After conv1 torch.Size([64, 256, 8, 8])
After conv1 torch.Size([64, 256, 8, 8])
After conv1 torch.Size([64, 256, 8, 8])
After conv1 torch.Size([64, 256, 8, 8])
After conv1 torch.Size([64, 256, 8, 8])
After conv1 torch.Size([64, 512, 8, 8])
After conv1 torch.Size([64, 512, 4, 4])
After conv1 torch.Size([64, 512, 4, 4])


tensor([[ 0.4784,  0.0289,  0.4199,  ...,  0.4338,  0.3973,  0.3966],
        [ 0.4784,  0.0289,  0.4199,  ...,  0.4338,  0.3973,  0.3966],
        [ 0.4784,  0.0289,  0.4199,  ...,  0.4338,  0.3973,  0.3966],
        ...,
        [ 0.4784,  0.0289,  0.4199,  ...,  0.4338,  0.3973,  0.3966],
        [ 0.4784,  0.0289,  0.4199,  ...,  0.4338,  0.3973,  0.3966],
        [ 0.4784,  0.0289,  0.4199,  ...,  0.4338,  0.3973,  0.3966]], device='cuda:0')

In [45]:
eval('3*4')

12